In [1]:
import os, sys, pathlib, glob, time, datetime, argparse, numpy as np, pandas as pd
import tensorflow.keras as keras
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Flatten, concatenate
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
import keras.backend as K
import matplotlib.pyplot as plt, seaborn as sns
from src import helper_models as hm, helper_data as hd, helper_tts as htts
from PIL import Image
from sklearn.metrics import classification_report
# import pdb
import numpy as np
import random


In [2]:
datapaths=["/local/kyathasr/plankifier/data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.04.28/"]
class_select=['dinobryon']
L=128


In [3]:
class_select,class_select_binary=hd.ReduceClasses(datapaths, class_select,'binary')	# Decide whether to use all available classes


datapaths: ['/local/kyathasr/plankifier/data/1_zooplankton_0p5x/training/zooplankton_trainingset_2020.04.28/']
classes from datapaths: ['cyclops', 'fragilaria', 'bosmina', 'daphnia', 'leptodora', 'conochilus', 'paradileptus', 'asterionella', 'chaoborus', 'dirt', 'diaphanosoma', 'fish', 'uroglena', 'unknown_plankton', 'asplanchna', 'ceratium', 'eudiaptomus', 'dinobryon', 'keratella_cochlearis', 'kellikottia', 'keratella_quadrata', 'rotifers', 'trichocerca', 'nauplius', 'filament', 'unknown']


In [4]:
class_select_binary

['dinobryon']

In [5]:
#class_select

In [6]:
class_select = [classes for classes in class_select if classes not in class_select_binary]

In [7]:
#class_select

In [8]:
#allClasses = list(set([ name for idata in range(len(datapaths)) for name in os.listdir(datapaths[idata]) if os.path.isdir(os.path.join(datapaths[idata], name))]))


In [9]:
#allClasses

In [10]:
#allClasses = [classes for classes in allClasses if classes not in class_select]


In [11]:
#allClasses

In [12]:
#set(class_select).issubset(allClasses)

In [13]:
#if class_select in allClasses: allClasses.remove(class_select)


In [14]:
#allClasses.remove(class_select)

In [15]:
#class_select=['dinobryon','conochilus']

In [16]:
df = pd.DataFrame()
dfA = pd.DataFrame()
dfB = pd.DataFrame()
names='/training_data/*.jp*g'
total_selected= 0
concatenated_list = []

In [18]:
		for c in class_select_binary:

			# Get names of images belonging to this class, from all the data paths
			BinaryclassImages = []
			for bidp in range(len(datapaths)):
				BinaryclassImages.extend( glob.glob(datapaths[bidp]+'/'+c+'/'+names) )

			# Create an empty dataframe for this class
			dfBClass=pd.DataFrame(columns=['classname','npimage'])

			print('class: {} ({})'.format(c, len(BinaryclassImages)))

			for bi,BinaryimageName in enumerate(BinaryclassImages):

				npimage,rescaled=hd.LoadImage(BinaryimageName,L)
				dfBClass.loc[bi] = [c,npimage]

			dfB=pd.concat([dfB,dfBClass], axis=0)

		dfB.npimage = dfB.npimage / 255.0 

class: dinobryon (2928)


In [19]:
	for c in class_select:

		# Get names of images belonging to this class, from all the data paths
		classImages = []
		for idp in range(len(datapaths)):
			classImages.extend( glob.glob(datapaths[idp]+'/'+c+'/'+names) )

		# Create an empty dataframe for this class
#		dfRandomClass=pd.DataFrame(columns=['classname'])        
#		print('class: {} ({})'.format(c, len(classImages)))

		random_classImages = np.random.choice(classImages, int(len(classImages)*0.45))
		print('class: {} ({})'.format(c, len(random_classImages)))

		concatenated_list=np.concatenate([concatenated_list,random_classImages])
        
#		total_selected+=len(random_classImages)
        
		Negative_class = np.random.choice(concatenated_list, len(BinaryclassImages))


class: cyclops (256)
class: fragilaria (95)
class: bosmina (21)
class: daphnia (234)
class: leptodora (88)
class: conochilus (28)
class: paradileptus (31)
class: asterionella (76)
class: chaoborus (2)
class: dirt (0)
class: diaphanosoma (490)
class: fish (71)
class: uroglena (392)
class: unknown_plankton (0)
class: asplanchna (144)
class: ceratium (100)
class: eudiaptomus (167)
class: keratella_cochlearis (45)
class: kellikottia (79)
class: keratella_quadrata (102)
class: rotifers (69)
class: trichocerca (109)
class: nauplius (406)
class: filament (91)
class: unknown (3)


In [20]:
negative_class_name='Not_' + ''.join(class_select_binary)               
# Create an empty dataframe for this class
dfAClass=pd.DataFrame(columns=['classname','npimage'])

print('class: {} ({})'.format(c, len(classImages)))

for i,imageName in enumerate(Negative_class):

	npimage,rescaled=hd.LoadImage(imageName,L)
	dfAClass.loc[i] = [negative_class_name,npimage]

dfA=pd.concat([dfA,dfAClass], axis=0)

dfA.npimage = dfA.npimage / 255.0 

class: unknown (7)


In [21]:
dfA

,classname,npimage
0,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
1,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
2,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
3,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
4,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
...,...,...
2923,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
2924,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
2925,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
2926,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."


In [22]:
dfB

,classname,npimage
0,dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
1,dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
2,dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
3,dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
4,dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
...,...,...
2923,dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
2924,dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
2925,dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
2926,dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."


In [23]:
dfA.shape

(2928, 2)

In [24]:
dfB.shape

(2928, 2)

In [25]:
df=pd.concat([dfA,dfB], axis=0)    # Concatenate Selected class and all other class


In [26]:
df

,classname,npimage
0,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
1,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
2,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
3,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
4,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
...,...,...
2923,dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
2924,dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
2925,dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
2926,dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."


In [27]:
df.shape

(5856, 2)

In [28]:
df.reset_index(drop=True)

,classname,npimage
0,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
1,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
2,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
3,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
4,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
...,...,...
5851,dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
5852,dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
5853,dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
5854,dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."


In [29]:
df

,classname,npimage
0,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
1,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
2,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
3,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
4,Not_dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
...,...,...
2923,dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
2924,dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
2925,dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."
2926,dinobryon,"[[[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0,..."


In [30]:
df['classname'].unique()

array(['Not_dinobryon', 'dinobryon'], dtype=object)